# Análisis exploratorio y descripción general de datos (RawData)

## Introducción

(Añadir descripción de cómo usar DataFiltering)

## 0. Cargar librerías

In [1]:
import pandas as pd
import numpy as np
import time as time
import dask.dataframe as dd
from tqdm import tqdm

import functions as fn

## 1. Cargar el dataset de casos positivos de COVID-19

In [3]:
url_cas = '../RawData/positivos_covid.csv'
df_cas = pd.read_csv(url_cas, sep = ';',  dtype = {'FECHA_CORTE': 'int32', 'DEPARTAMENTO': 'category',
                                                   'PROVINCIA': 'category', 'Distrito': 'category',
                                                   'METODODX': 'category', 'SEXO': 'category'})
del url_cas
df_cas

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
0,20220114,LIMA,LIMA,SAN MARTIN DE PORRES,PR,25.0,MASCULINO,20201217.0,150135.0,24662153.0
1,20220114,ICA,PISCO,PISCO,PR,20.0,FEMENINO,20200822.0,110501.0,24662175.0
2,20220114,HUANUCO,HUANUCO,HUANUCO,PR,22.0,FEMENINO,20200729.0,100101.0,24662197.0
3,20220114,ANCASH,SANTA,SANTA,AG,18.0,FEMENINO,20210630.0,21808.0,24662204.0
4,20220114,ANCASH,SANTA,NUEVO CHIMBOTE,AG,17.0,MASCULINO,20210404.0,21809.0,24662207.0
...,...,...,...,...,...,...,...,...,...,...
2562530,20220114,LIMA,LIMA,LIMA,PCR,23.0,MASCULINO,20200423.0,150101.0,NaN
2562531,20220114,LIMA,LIMA,CHORRILLOS,PCR,47.0,FEMENINO,20210122.0,150108.0,NaN
2562532,20220114,CALLAO,CALLAO,CALLAO,PCR,52.0,MASCULINO,20210911.0,70101.0,NaN
2562533,20220114,LIMA,LIMA,LIMA,PCR,50.0,FEMENINO,20200518.0,150101.0,NaN


In [4]:
fn.missing_values(df_cas)

,index,percent_missing
0,FECHA_CORTE,0.000000
1,DEPARTAMENTO,0.000000
2,PROVINCIA,0.000000
3,DISTRITO,0.000000
4,METODODX,0.000000
5,EDAD,0.013541
6,SEXO,0.000039
7,FECHA_RESULTADO,0.078945
8,UBIGEO,4.927738
9,id_persona,1.811058


In [5]:
df_cas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2562535 entries, 0 to 2562534
Data columns (total 10 columns):
 #   Column           Dtype   
---  ------           -----   
 0   FECHA_CORTE      int32   
 1   DEPARTAMENTO     category
 2   PROVINCIA        category
 3   DISTRITO         object  
 4   METODODX         category
 5   EDAD             float64 
 6   SEXO             category
 7   FECHA_RESULTADO  float64 
 8   UBIGEO           float64 
 9   id_persona       float64 
dtypes: category(4), float64(4), int32(1), object(1)
memory usage: 119.8+ MB


In [34]:
# FECHA_CORTE
contarFechaCorte = df_cas['FECHA_CORTE'].value_counts().reset_index()
fechaCorte = pd.to_datetime(contarFechaCorte['index'], format = '%Y%m%d').dt.date
print(f'Fecha de corte: {fechaCorte[0]}')
del contarFechaCorte, fechaCorte

Fecha de corte: 2022-01-14


In [83]:
# DEPARTAMENTO
totalCases = df_cas['DEPARTAMENTO'].value_counts().sort_index().sum()
print(f'Total de casos positivos: {totalCases}')
del totalCases

df_cas['DEPARTAMENTO'].value_counts().sort_index()

Total de casos positivos: 2562535


AMAZONAS           34329
ANCASH             90283
APURIMAC           28130
AREQUIPA          132459
AYACUCHO           37460
CAJAMARCA          73240
CALLAO            115273
CUSCO              89495
HUANCAVELICA       17595
HUANUCO            39732
ICA                69051
JUNIN              96882
LA LIBERTAD       103214
LAMBAYEQUE         75274
LIMA             1156394
LORETO             46674
MADRE DE DIOS      14539
MOQUEGUA           32664
PASCO              17584
PIURA             105525
PUNO               46376
SAN MARTIN         51840
TACNA              35481
TUMBES             20090
UCAYALI            32951
Name: DEPARTAMENTO, dtype: int64

In [75]:
# PROVINCIA
df_cas['PROVINCIA'].value_counts().sort_index()

ABANCAY          12744
ACOBAMBA          1379
ACOMAYO            688
AIJA               366
ALTO AMAZONAS     5228
                 ...  
YAULI             2897
YAUYOS             737
YUNGAY            1638
YUNGUYO           1102
ZARUMILLA         3250
Name: PROVINCIA, Length: 197, dtype: int64

In [84]:
# DISTRITO
df_cas['DISTRITO'].value_counts().sort_index()

ABANCAY                    9790
ABELARDO PARDO LEZAMETA      28
ACARI                       503
ACAS                         10
ACCHA                       172
                           ... 
ZEPITA                      182
ZORRITOS                    894
ZURITE                       87
ZUÑIGA                       11
ÑAHUIMPUQUIO                  4
Name: DISTRITO, Length: 1698, dtype: int64

In [88]:
# METODODX
df_cas['METODODX'].value_counts().sort_index()

AG     831122
PCR    775547
PR     955866
Name: METODODX, dtype: int64

In [105]:
# EDAD
edadCasos = df_cas['EDAD'].value_counts()

print('Edad (años): no_casos \nDesde la edad más común a la menos común.\n')
lst = []

for idx, edad in enumerate(edadCasos):
    index = edadCasos.index
    lst.append(f'{index[idx]}: {edad}   ')
    
chunks = [lst[x:x+25] for x in range(0, len(lst), 25)]
df = pd.DataFrame.from_records(chunks).transpose()

print(df.to_string(index=False, header=False))

del edadCasos, chunks, df, lst, edad, idx, index

Edad (años): no_casos 
Desde la edad más común a la menos común.

32.0: 63924    48.0: 43950    68.0: 18581    82.0: 6322    100.0: 119   
33.0: 63247    49.0: 43208    17.0: 18139     7.0: 6225     101.0: 89   
28.0: 62500    50.0: 42058    69.0: 16568     1.0: 6224     102.0: 53   
29.0: 61213    51.0: 40299    16.0: 16350    83.0: 5850     103.0: 33   
31.0: 60681    22.0: 39814    70.0: 16032     6.0: 5474     104.0: 19   
27.0: 60523    52.0: 39803    71.0: 14886    84.0: 5241     120.0: 19   
30.0: 59869    53.0: 38973    15.0: 14302     5.0: 4978     105.0: 15   
34.0: 59334    54.0: 38879    72.0: 13898     2.0: 4795     106.0: 10   
26.0: 57975    55.0: 37151    73.0: 13520    85.0: 4680      108.0: 7   
35.0: 57339    21.0: 36225    14.0: 12145     4.0: 4643      121.0: 5   
38.0: 56742    56.0: 35151    74.0: 12120     3.0: 4507      107.0: 4   
25.0: 55989    57.0: 33421    75.0: 11655    86.0: 4072      119.0: 4   
37.0: 55897    58.0: 32059    76.0: 10651    87.0: 3527   

In [106]:
# SEXO
df_cas['SEXO'].value_counts().sort_index()

FEMENINO     1265001
MASCULINO    1297533
Name: SEXO, dtype: int64

In [107]:
# FECHA_RESULTADO
df_cas['FECHA_RESULTADO'].head()

0    20201217.0
1    20200822.0
2    20200729.0
3    20210630.0
4    20210404.0
Name: FECHA_RESULTADO, dtype: float64

In [108]:
# UBIGEO
df_cas['UBIGEO'].head()

0    150135.0
1    110501.0
2    100101.0
3     21808.0
4     21809.0
Name: UBIGEO, dtype: float64

In [109]:
# id_persona
df_cas['id_persona'].head()

0    24662153.0
1    24662175.0
2    24662197.0
3    24662204.0
4    24662207.0
Name: id_persona, dtype: float64

In [110]:
del df_cas

## 2. Cargar el dataset de fallecidos por COVID-19

In [5]:
url_fal = '../RawData/fallecidos_covid.csv'
df_fal = pd.read_csv(url_fal, sep = ";")

df_fal.head()

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,id_persona
0,20220106,20210423,41,MASCULINO,Criterio virolÃ³gico,LIMA,LIMA,VILLA MARIA DEL TRIUNFO,150143,24769779.0
1,20220106,20210607,31,FEMENINO,Criterio SINADEF,CALLAO,CALLAO,CALLAO,070101,24742499.0
2,20220106,20210708,27,FEMENINO,Criterio SINADEF,LIMA,LIMA,CARABAYLLO,150106,24742552.0
3,20220106,20210518,62,MASCULINO,Criterio virolÃ³gico,JUNIN,TARMA,TARMA,120701,24633504.0
4,20220106,20210519,74,MASCULINO,Criterio virolÃ³gico,LIMA,LIMA,SANTA ANITA,150137,24633522.0


In [5]:
df_fal[df_fal.id_persona == 24907449.0]

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,id_persona
232,20220106,20200316,26,FEMENINO,Criterio clÃ­nico,AYACUCHO,HUAMANGA,AYACUCHO,050101,24907449.0


In [6]:
fn.missing_values(df_fal)

,index,percent_missing
0,FECHA_CORTE,0.000000
1,FECHA_FALLECIMIENTO,0.000000
2,EDAD_DECLARADA,0.000000
3,SEXO,0.000000
4,CLASIFICACION_DEF,0.000000
5,DEPARTAMENTO,0.000000
6,PROVINCIA,0.002463
7,DISTRITO,0.002463
8,UBIGEO,0.000000
9,id_persona,1.700652


In [4]:
df_fal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202981 entries, 0 to 202980
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   FECHA_CORTE          202981 non-null  int64  
 1   FECHA_FALLECIMIENTO  202981 non-null  int64  
 2   EDAD_DECLARADA       202981 non-null  int64  
 3   SEXO                 202981 non-null  object 
 4   CLASIFICACION_DEF    202981 non-null  object 
 5   DEPARTAMENTO         202981 non-null  object 
 6   PROVINCIA            202976 non-null  object 
 7   DISTRITO             202976 non-null  object 
 8   UBIGEO               202981 non-null  object 
 9   id_persona           199529 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 15.5+ MB


### Analizar cada variable del dataset de fallecidos

#### 1.1. FECHA_CORTE

In [6]:
print('Fecha de corte:')
df_fal['FECHA_CORTE'].value_counts().sort_index()

Fecha de corte:


20220106    202981
Name: FECHA_CORTE, dtype: int64

Todos los valores son iguales. La misma fecha de corte para n número de casos, variable no necesaria.

#### 1.2. FECHA_FALLECIMIENTO

In [7]:
# Fecha del primer fallecido reportado
date_a = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d').dt.date
date_a = date_a.min()
print(f'Fecha del primer fallecido reportado: {date_a}')
del date_a

# Fecha del último fallecido reportado
date_b = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d').dt.date
date_b = date_b.max()
print(f'Fecha del último fallecido reportado: {date_b}')
del date_b

Fecha del primer fallecido reportado: 2020-03-03
Fecha del último fallecido reportado: 2022-01-06


#### 1.3. EDAD_DECLARADA

In [8]:
fal_edad = df_fal['EDAD_DECLARADA'].value_counts()

print('Edad (años): no_casos \nDesde la edad más común de fallecimientos a la menos común.\n')
lst = []

for idx, edad in enumerate(fal_edad):
    index = fal_edad.index
    lst.append(f'{index[idx]}: {edad}')
    
chunks = [lst[x:x+20] for x in range(0, len(lst), 20)]
df = pd.DataFrame.from_records(chunks).transpose()

print(df.to_string(index=False, header=False))

del fal_edad, chunks, df, lst, edad, idx, index

Edad (años): no_casos 
Desde la edad más común de fallecimientos a la menos común.

66: 5513 80: 4150 48: 2225 34: 642   1: 149 102: 40
67: 5414 79: 4028 89: 2090 33: 630  20: 113   5: 40
65: 5389 58: 4027 47: 2029 95: 566  19: 111   6: 38
68: 5360 57: 4009 90: 1876 32: 557 100: 106 103: 22
70: 5260 82: 3860 46: 1856 96: 459   18: 89 104: 16
72: 5201 81: 3831 45: 1781 31: 435    2: 83  106: 9
73: 5187 56: 3719 44: 1676 30: 423   17: 77  108: 6
69: 5166 55: 3698 43: 1506 29: 346   16: 75  105: 5
64: 5099 83: 3567 91: 1447 27: 318  101: 71  107: 3
74: 5089 54: 3443 42: 1359 28: 313    4: 63   -9: 3
71: 5069 84: 3406 92: 1355 97: 306   15: 56  113: 2
63: 5060 85: 3216 41: 1271 26: 267   14: 53  110: 2
61: 5048 53: 3114 40: 1097 98: 246   11: 52  109: 2
75: 4946 86: 2851 39: 1066 25: 243    8: 51  117: 1
62: 4915 52: 2813 93: 1019 24: 204    3: 51  114: 1
76: 4811 51: 2710  38: 976 99: 192    9: 51    None
60: 4602 87: 2673  37: 835 23: 190   10: 50    None
77: 4599 50: 2568  94: 765 21: 1

Existen valores negativos, necesario correguir. Edad mínima 0 años y edad máxima 117 años.

#### 1.4. SEXO

In [9]:
fal_sexo = df_fal['SEXO'].value_counts().sort_index()

for idx, sex in enumerate(fal_sexo):
    index = fal_sexo.index
    print(f'{index[idx]}: {sex}')

del fal_sexo, sex, idx, index

.: 3
FEMENINO: 73775
MASCULINO: 129203


Hay puntos en lugar de vacíos.

#### 1.5. CLASIFICACION_DEF

In [10]:
fal_cla = df_fal['CLASIFICACION_DEF'].value_counts().sort_index()

for idx, clase in enumerate(fal_cla):
    index = fal_cla.index
    print(f'{index[idx]}: {clase}')

del fal_cla, clase, idx, index

Criterio SINADEF: 61259
Criterio clÃ­nico: 10964
Criterio investigaciÃ³n EpidemiolÃ³gica: 16259
Criterio nexo epidemiolÃ³gico: 3014
Criterio radiolÃ³gico: 6845
Criterio serolÃ³gico: 44944
Criterio virolÃ³gico: 59696


#### 1.6. DEPARTAMENTO

In [11]:
fal_dep = df_fal['DEPARTAMENTO'].value_counts().sort_index()

for idx, dep in enumerate(fal_dep):
    index = fal_dep.index
    print(f'{index[idx]}: {dep}')
print(f'\nTotal fallecidos: {fal_dep.sum()}')

del fal_dep, dep, idx, index

AMAZONAS: 1288
ANCASH: 6777
APURIMAC: 1525
AREQUIPA: 9766
AYACUCHO: 2165
CAJAMARCA: 4199
CALLAO: 10127
CUSCO: 4838
HUANCAVELICA: 1184
HUANUCO: 2728
ICA: 8621
JUNIN: 7110
LA LIBERTAD: 10408
LAMBAYEQUE: 8726
LIMA: 89576
LORETO: 4286
MADRE DE DIOS: 782
MOQUEGUA: 1535
PASCO: 1053
PIURA: 12307
PUNO: 4276
SAN MARTIN: 3039
TACNA: 1980
TUMBES: 1597
UCAYALI: 3088

Total fallecidos: 202981


#### 1.7. PROVINCIAS

In [12]:
fal_pro = df_fal['PROVINCIA'].value_counts().sort_index()
print(f'Igual que la variable departamento pero con {len(fal_pro)} provincias.')
del fal_pro

Igual que la variable departamento pero con 196 provincias.


#### 1.8. DISTRITO

In [13]:
fal_dis = df_fal['DISTRITO'].value_counts().sort_index()
print(f'Igual que la variable departamento pero con {len(fal_dis)} distritos.')
del fal_dis

Igual que la variable departamento pero con 1648 distritos.


#### 1.9. UBIGEO

In [14]:
fal_ubi = df_fal['UBIGEO'].value_counts().sort_index()
print(f'{len(fal_ubi)} códigos de UBIGEO, para localizar zonas.')
del fal_ubi

1783 códigos de UBIGEO, para localizar zonas.


#### 1.10. id_persona

In [15]:
df_fal['id_persona'] = df_fal['id_persona'].fillna(0).apply(np.int64)

print('ID únicos para identificar a cada persona: ')
print(df_fal['id_persona'].head().to_string(header=False))

ID únicos para identificar a cada persona: 
0    24769779
1    24742499
2    24742552
3    24633504
4    24633522


In [7]:
del df_fal

## 3. Cargar el dataset de vacunados contra COVID-19

### Cargar usando dask, debido al tamaño del csv

In [9]:
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'

start = time.time()
df_vac = dd.read_csv(url_vac, sep = ",", dtype = {'fecha_vacunacion':'int32', 'id_eess': 'int32',
                                                  'dosis': 'category','id_centro_vacunacion': 'category',
                                                  'id_vacuna': 'category','id_grupo_riesgo': 'category', 
                                                  'edad':'float64'})
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

del url_vac, start, end
df_vac.head()

Read csv with dask:  0.2881488800048828 sec


,id_persona,id_vacunados_covid19,fecha_vacunacion,id_eess,id_centro_vacunacion,id_vacuna,id_grupo_riesgo,dosis,edad
0,17289500,17135722,20210623,5468,0,4,54,1,69.0
1,19180697,20881090,20211210,12622,0,6,67,1,44.0
2,19374477,20882754,20211221,2102,0,5,67,3,41.0
3,19152824,20884738,20210619,2297,674,4,67,2,41.0
4,23501162,31058327,20211211,2343,121,5,54,3,60.0


In [8]:
df_vac[df_vac.id_persona == 24907449].compute()

,id_persona,id_vacunados_covid19,fecha_vacunacion,id_eess,id_centro_vacunacion,id_vacuna,id_grupo_riesgo,dosis,edad
72096,24907449,42401625,20211031,17674,677,6,71,2,27.0
1315627,24907449,42316642,20211010,770,677,6,71,1,27.0


### Porcentaje de datos perdidos de cada variable

In [10]:
fn.missing_values(df_vac)

,index,percent_missing
0,0,0.00000
1,1,0.00000
2,2,0.00000
3,3,0.00000
4,4,0.00000
5,5,0.00000
6,6,0.00000
7,7,0.00000
8,8,0.00803


### Analizar cada variable del dataset de vacunados

#### 2.1. id_persona

In [ ]:
print('ID únicos para identificar a cada persona:')
df_vac['id_persona'].head()

ID únicos para identificar a cada persona:


0    7525299
1    5535776
2    8837098
3    5222654
4    5168206
Name: id_persona, dtype: int64

#### 2.2. id_vacunados_covid19

In [ ]:
print('ID únicos para identificar a cada vacunado (como un contador de cada vacuna aplicada):')
df_vac['id_vacunados_covid19'].head()

ID únicos para identificar a cada vacunado (como un contador de cada vacuna aplicada):


0    22315832
1    16020164
2    25810259
3    15274372
4    15456307
Name: id_vacunados_covid19, dtype: int64

#### 2.3. fecha_vacunacion

In [ ]:
date_a = df_vac['fecha_vacunacion'].min().compute()
date_a = pd.to_datetime(date_a, format = '%Y%m%d')

date_b = df_vac['fecha_vacunacion'].max().compute()
date_b = pd.to_datetime(date_b, format = '%Y%m%d')

print(f'Fecha de inicio de vacunación: {date_a} \nFecha última vacunación: {date_b}')
del date_a, date_b

Fecha de inicio de vacunación: 2020-04-27 00:00:00 
Fecha última vacunación: 2022-01-04 00:00:00


#### 2.4. id_eess

In [ ]:
print(f'id_iess\tnumber')
df_vac.id_eess.value_counts().compute().sort_index()

id_iess	number


33          36
49         220
51        1419
52        2177
53          63
         ...  
28772      731
28780       23
28791      611
28806    11330
28820      691
Name: id_eess, Length: 7884, dtype: int64

#### 2.5. id_centro_vacunacion	

In [ ]:
print(f'id_centro_vacunacion\tnumber')
df_vac.id_centro_vacunacion.value_counts().compute().sort_index()

id_centro_vacunacion	number


0       14741687
1           1924
10         60439
100         3299
104          544
          ...   
2705           2
2998           1
567            1
1865           1
4164           1
Name: id_centro_vacunacion, Length: 3441, dtype: int64

#### 2.6. id_vacuna

In [ ]:
print(f'id_vacuna\tnumber')
df_vac.id_vacuna.value_counts().compute().sort_index()

id_vacuna	number


1     3167931
2      371456
3      396814
4      700871
5    27088772
6    18621506
Name: id_vacuna, dtype: int64

#### 2.7. id_grupo_riesgo

In [ ]:
df_vac.id_grupo_riesgo.value_counts().compute().sort_index()

0        3909
11     227006
12     334538
17    1535775
22     111776
23     417903
26       4044
36      40384
4       30392
40     345466
45      16996
47      18405
50      17525
51       9826
52       1183
53      18355
54    9504978
56      29077
57     106243
58     110676
59       3063
6        2297
60      30110
61      11345
62    6048890
63      36509
64      70757
65      44765
66     255997
67    7498829
68    8157747
69     130656
7      316962
70    1671237
71    8413330
72       4967
73       4814
74       3228
75       4688
76      17600
77    4732210
78        943
79         82
1         207
24       1660
Name: id_grupo_riesgo, dtype: int64

#### 2.8. dosis

In [ ]:
print(f'dosis\tnumber')
df_vac.dosis.value_counts().compute()

dosis	number


1    24511772
2    21611587
3     4223869
4         116
5           6
Name: dosis, dtype: int64

#### 2.9. edad

In [ ]:
df_vac.edad.value_counts().compute().sort_index()

0.0      3
1.0      2
2.0      4
3.0      2
4.0      7
        ..
141.0    2
143.0    2
146.0    2
147.0    1
224.0    2
Name: edad, Length: 136, dtype: int64